In [14]:
%pip install ipywidgets tqdm PyMuPDF unidecode nltk contractions spacy scipy neo4j networkx matplotlib transformers huggingface_hub[hf_xet] streamlit chromadb sentence_transformers
%pip install torch --index-url https://download.pytorch.org/whl/cu126
# python -m spacy download en_core_web_sm

  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached scikit_learn-1.7.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-an

  You can safely remove it manually.


Looking in indexes: https://download.pytorch.org/whl/cu126
Note: you may need to restart the kernel to use updated packages.


In [7]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(dotenv_path=r"C:\Users\Asus\Downloads\GraphRAG Project\Secrets\.env")

# Retrieve the Hugging Face token
HF_TOKEN = os.getenv("HF_TOKEN")
if not HF_TOKEN:
    raise ValueError("HF_TOKEN not found in environment variables")
else:
    print("HF_TOKEN loaded successfully")

HF_TOKEN loaded successfully


In [8]:
import fitz  # PyMuPDF
from tqdm.notebook import tqdm

def extract_text_from_pdf(filepath):
    doc = fitz.open(filepath)
    full_text = ""
    for page in tqdm(doc):
        full_text += page.get_text()
    doc.close()
    return full_text

text = extract_text_from_pdf(r"C:\\Users\\Asus\\Downloads\\GraphRAG Project\\Data\\Ebay_user_agreement.pdf")
text

  0%|          | 0/31 [00:00<?, ?it/s]

'User Agreement \n1. Introduction \nThis User Agreement, the Mobile Application Terms of Use, and all policies and \nadditional terms posted on and in our sites, applications, tools, and services (collectively \n"Services") set out the terms on which eBay offers you access to and use of our \nServices. You can find an overview of our policies here. The Mobile Application Terms of \nUse, all policies, and additional terms posted on and in our Services are incorporated \ninto this User Agreement. You agree to comply with all terms of this User Agreement \nwhen accessing or using our Services. \nThe entity you are contracting with is: eBay Inc., 2025 Hamilton Ave., San Jose, CA \n95125, if you reside in the United States; eBay (UK) Limited, 1 More London Place, \nLondon, SE1 2AF, United Kingdom, if you reside in the United Kingdom; eBay GmbH, \nAlbert-Einstein-Ring 2-6, 14532 Kleinmachnow, Germany, if you reside in the European \nUnion; eBay Canada Limited, 240 Richmond Street West, 2nd F

In [9]:
import re
from unidecode import unidecode
import contractions

def clean_text_debug(text):
    removed = {}
    
    # Protect common abbreviations (customize as needed)
    text = text.replace("U.S.", "___US___")
    text = text.replace("U.K.", "___UK___")

    # Expand contractions safely
    text = contractions.fix(text)

    # Restore the protected terms
    text = text.replace("___US___", "U.S.")
    text = text.replace("___UK___", "U.K.")

    # Normalize Unicode (e.g., smart quotes → straight quotes)
    # text = unidecode(text)
    
    # Store and remove URLs
    urls = re.findall(r'http\S+|www\S+|https\S+', text)
    removed['urls'] = urls
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Store and remove emails
    emails = re.findall(r'\S+@\S+', text)
    removed['emails'] = emails
    text = re.sub(r'\S+@\S+', '', text)

    # Optionally mark paragraph or section breaks for GraphRAG chunking
    text = re.sub(r'\n{2,}', '\n---\n', text)  # Replace multiple newlines with delimiter
    text = re.sub(r'(?<=\w)\n(?=\w)', ' ', text)  # Join broken lines within paragraphs

    # Store and remove unwanted non-alphanumeric characters (allowing useful ones)
    pattern_to_keep =   r"[^\w\s.,;:()?!\-/\'“”’\"$§]"
    non_alpha = re.findall(pattern_to_keep, text)
    removed['non_alpha'] = non_alpha
    text = re.sub(pattern_to_keep, '', text)

    # Normalize excessive whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text, removed

cleaned_text, removed_items = clean_text_debug(text)

for category, items in removed_items.items():
    print(f"\nRemoved {category} ({len(items)} items):")
    print(items)  # Show only first 10 for brevity

# Save cleaned text or proceed to chunking/NER
with open(r"C:\\Users\\Asus\\Downloads\\GraphRAG Project\\Data\\cleaned_ebay_user_agreement.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)
    print("Cleaned text saved to cleaned_ebay_user_agreement.txt")


Removed urls (6 items):
['https://www.namadr.com/resources/rules-fees-forms/.', 'https://www.namadr.com/content/uploads/2024/09/Revised-Comprehensive-D', 'www.eBay.com.', 'www.ebay.com.', 'www.eBay.com.', 'www.adr.org.']

Removed emails (1 items):
['DisputeNotice@eBay.com']

Removed non_alpha (172 items):
['#', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '&', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '\u200b', '●', '

In [10]:
print(cleaned_text)

User Agreement 1. Introduction This User Agreement, the Mobile Application Terms of Use, and all policies and additional terms posted on and in our sites, applications, tools, and services (collectively "Services") set out the terms on which eBay offers you access to and use of our Services. You can find an overview of our policies here. The Mobile Application Terms of Use, all policies, and additional terms posted on and in our Services are incorporated into this User Agreement. You agree to comply with all terms of this User Agreement when accessing or using our Services. The entity you are contracting with is: eBay Inc., 2025 Hamilton Ave., San Jose, CA 95125, if you reside in the United States; eBay (UK) Limited, 1 More London Place, London, SE1 2AF, United Kingdom, if you reside in the United Kingdom; eBay GmbH, Albert-Einstein-Ring 2-6, 14532 Kleinmachnow, Germany, if you reside in the European Union; eBay Canada Limited, 240 Richmond Street West, 2nd Floor Suite 02-100, Toronto,

In [11]:
len(text)

68448

In [12]:
len(cleaned_text)

67082

In [13]:
# print text not in cleaned_ebay_user_agreement.txt from cleaned_ebay.txt
with open(r"C:\\Users\\Asus\\Downloads\\GraphRAG Project\\Data\\cleaned_ebay_user_agreement.txt", "r", encoding="utf-8") as f:
    cleaned_ebay_user_agreement = f.read()
# Find text in cleaned_ebay not in cleaned_ebay_user_agreement
unique_text_ebay = set(text.split()) - set(cleaned_ebay_user_agreement.split())
print("Unique text in text not in cleaned_ebay_user_agreement:")
print(unique_text_ebay)
print("\n" + "="*50 + "\n")
# Find text in cleaned_ebay not in cleaned_ebay_user_agreement
unique_text_ebay = set(cleaned_ebay_user_agreement.split()) - set(text.split())
print("Unique text in cleaned_ebay_user_agreement not in text:")
print(unique_text_ebay)

Unique text in text not in cleaned_ebay_user_agreement:
{'Arbitrate\u200b', 'Relief\u200b', 'low‐cost', 'Procedure\u200b', '&', "don't", 'Trial\u200b', 'Costs\u200b', 'arbitrator.\u200b', 'Severability\u200b', 'e‐mail', 'Arbitration\u200b', 'Arbitrate.\u200b', 'DisputeNotice@eBay.com', 'eBay.\u200b', 'https://www.namadr.com/resources/rules-fees-forms/.', '1.\u200b', '6.\u200b', 'https://www.namadr.com/content/uploads/2024/09/Revised-Comprehensive-D', "doesn't", 'account.\u200b', '●\u200b', '4.\u200b', 'cause', 'Resolution\u200b', '10.\u200bFuture', 'Arbitration.\u200b', 'offer.\u200b', '2.\u200b', 'Procedures\u200b', "didn't", '3.\u200b', 'Arbitration").\u200b', 'good‐faith', 'Arbitrator\u200b', '7.\u200b', 'possible.\u200b', 'www.adr.org.', '9.\u200b', "can't", '#18-', '5.\u200b', 'www.eBay.com.', 'administrator.\u200b', 'www.ebay.com.', 'Demand.\u200b', '8.\u200b', "there's"}


Unique text in cleaned_ebay_user_agreement not in text:
{'10.Future', '18-', 'goodfaith', 'offer.', 'Proced

In [14]:
temp = "YOU.S"
# temp = "YOU.S.C."

idx = cleaned_text.find(temp)
if idx != -1:
    # Replace the contraction with its expanded form
    print(cleaned_text[idx:])
else:
    print(f"'{temp}' not found in cleaned_text")

'YOU.S' not found in cleaned_text


In [15]:
with open(r"C:\\Users\\Asus\\Downloads\\GraphRAG Project\\Data\\cleaned_ebay_user_agreement.txt", "r", encoding="utf-8") as f:
    cleaned_text = f.read()

In [16]:
cleaned_text

'User Agreement 1. Introduction This User Agreement, the Mobile Application Terms of Use, and all policies and additional terms posted on and in our sites, applications, tools, and services (collectively "Services") set out the terms on which eBay offers you access to and use of our Services. You can find an overview of our policies here. The Mobile Application Terms of Use, all policies, and additional terms posted on and in our Services are incorporated into this User Agreement. You agree to comply with all terms of this User Agreement when accessing or using our Services. The entity you are contracting with is: eBay Inc., 2025 Hamilton Ave., San Jose, CA 95125, if you reside in the United States; eBay (UK) Limited, 1 More London Place, London, SE1 2AF, United Kingdom, if you reside in the United Kingdom; eBay GmbH, Albert-Einstein-Ring 2-6, 14532 Kleinmachnow, Germany, if you reside in the European Union; eBay Canada Limited, 240 Richmond Street West, 2nd Floor Suite 02-100, Toronto

In [17]:
len(cleaned_text.split())

10879

In [18]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(cleaned_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [19]:
len(sentences)

314